# 文字列の処理

Juliaではダブルクオーテーション `"` でくくった内容が、文字列として扱われる。
文字列の型は `String` であり、多数の文字の集合である。
ここでは代表的な文字列処理の方法を紹介する。
なお、この章の説明は、データ構造に関する章を読み終わっていることを前提としている。

また、すでに、[非常によくまとまった入門ガイド](https://github.com/bicycle1885/Julia-Tutorial/blob/master/Julia高速チュートリアル.ipynb)があるので、詳細はそちらを参照して欲しい。

## 文字列の基本的な扱い方

### 文字列の定義と抽出

文字列を定義するには、ダブルクオーテーションで内容物をくくればよい。
文字列は数値と同様、変数に代入することができる。
その変数の型は `String` になる。

In [1]:
s = "ABCDEFG"

"ABCDEFG"

In [2]:
typeof(s)

String

文字列から一部を切り出すには、配列と同じく末尾に `[]` を付加し、取り出す範囲を指定する。
範囲の指定は `開始位置:終了位置` の形式でしている。
もし終了位置に、キーワード `end` を使うなら、それは文字列の末端を指定したことと同じである。
また、開始位置と終了位置の両方を省略して `:` のみを使うと、それは文字列全体を指定したのと同じ意味になる（`1:end`）。
抜き出した文字列もまた、`String` である。

In [3]:
# 3文字目から5文字目を抜き出す
s[3:5]

"CDE"

1文字だけ抜き出す場合は注意を要する。
もし開始位置を範囲でなく1つの値にするなら、得られる値は文字列ではなく、文字 `Char` になる。

In [4]:
# 3文字目の「文字」を抜き出す
s[3]

'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)

もし1文字だけの「文字列」を得たいなら、範囲表記を使い、開始と終了の位置を同じにすればよい。

In [5]:
# 3文字目を「文字列」として抜き出す
s[3:3]

"C"

### 文字列の結合

二つの文字列を結合するには、演算子 `*` を用いる（Perlでは `.` に相当する）。

In [6]:
t = s * "HIJKL"

"ABCDEFGHIJKL"

### 数値を文字列に変換する（string関数）

数値を文字列に変換するには、`string()` 関数を用いる。
この関数は複数の引数を取り、それらを区切りなしで結合する。
これは、実際には数値以外のスカラも文字列に変換することができる。

In [7]:
# 整数の変換
string(1000)

"1000"

In [8]:
# 実数の変換
string(3.14)

"3.14"

In [9]:
# 複数の要素を同時に変換する
string(1000,3.14)

"10003.14"

In [10]:
# 間に文字列を挟むことができる
string(1000," ",3.14)

"1000 3.14"

### 数値を文字列に変換する（@sprintfマクロ）

数値を精密に（小数点以下の桁数などを制御して）文字列に変換するには、いろいろな方法があるが、`@sprintf` が簡単である。
これはC言語以下、各種言語でサポートされている `sprintf` の挙動を真似るものである。

In [11]:
# 変数 x を定義する
x = 3.1415926

# 上記の x を小数点以下2桁までに揃える
y = @sprintf("%.2f",x)

"3.14"

### 変数を文字列内に展開する

Juliaでは、Perlと同じく、文字列中で変数を展開することができる（これを内挿 interpolation という）。
文字列内の変数は `$()` でくくる。
数値の変換フォーマットは、Juliaに一任する。

In [12]:
# 内挿する
"Pi is $(x)."

"Pi is 3.1415926."

In [13]:
# @sprintfも使える
"Pi is $(@sprintf("%.2f",x))."

"Pi is 3.14."

### 特殊文字

文字列の中で、`\` で始まる記号は、特別な意味を持つ文字列に置き換わる。
以下に一部を示す。

|記号     |置き換わる記号または意味|
|:-------|:-------------------|
|`\n`     |改行|
|`\t`     |タブ|
|`\$`     |ドル記号そのもの|
|`\"`     |ダブルクオーテーションそのもの|



In [14]:
# ためしに全部入れてみる
print("I have \$100\tafter a tab.\nThe next line.\t\"double quotation\"")

I have $100	after a tab.
The next line.	"double quotation"

## 文字列関数

Juliaには多数の文字列関数が用意されている。
標準状態で使えるものは、[マニュアル](https://docs.julialang.org/en/stable/stdlib/strings/)に記載がある。
ここではそのごく一部を紹介する。

### 文字列の長さを得る

関数 `length()` は文字列の長さを返す。

In [15]:
length("abcdefg")

7

### 文字列の反転

関数 `reverse()` を使う。

In [16]:
reverse("abcdefg")

"gfedcba"

### 先頭と末尾のスペースを除去する

関数 `strip()` は、文字列の先頭と末尾を埋めるスペースを除去する。同様に `lstrip()` は左端（先頭）のスペースのみを、`rstrip()` は右端（末尾）のスペースのみを除去する。

In [17]:
strip(" a  b  c  ")

"a  b  c"

### 文字列を区切り文字で分割する

文字列の分割には `split()` が利用できる。
区切り文字のデフォルトは、1つ以上の連続するスペースである（先頭と末尾のスペースは無視される）。
この挙動は awk のフィールド分割（`$1`や`$2`）と似ている。
区切られた部分文字列は、ひとつの配列（ベクトル）に入れられて戻される。
以下に具体的な使い方を示す。

In [18]:
# 文字列
str = " a  b  c "

# デフォルトの動作
split(str)

3-element Array{SubString{String},1}:
 "a"
 "b"
 "c"

In [19]:
# 第2引数で区切り文字を指定する
# この場合、1個のスペースを区切りにしたので、大量の空文字列が発生する
split(str," ")

7-element Array{SubString{String},1}:
 "" 
 "a"
 "" 
 "b"
 "" 
 "c"
 "" 

In [20]:
# オプション引数 keep=false とすると、分割後の空文字列を除去できる
# keep=true なら、空文字列をキープする
split(str," ",keep=false)

3-element Array{SubString{String},1}:
 "a"
 "b"
 "c"

### 文字列配列の結合

関数 `join()` は、文字列の結合を行う（Perlのjoinと同じ）；つまり上の `split()` の逆を行う。

In [21]:
# 文字列配列を用意する
ar = split(str)
# スペースで結合する
join(ar," ")

"a b c"

In [22]:
# 第3引数は、最後の要素を結合する時にだけ使われる
# 英語の文脈でよく出てくる表現を作るために使える
namelist = ["Yamada", "Tanaka", "Yamaguchi"]
join(namelist,", ",", and ")

"Yamada, Tanaka, and Yamaguchi"

### 左寄せと右寄せ

文字列を左寄せ（末尾にスペースを詰める）には `lpad()` を、右寄せ（先頭にスペースを詰める）には `rpad()` を利用する。

In [23]:
# 左寄せで10文字に揃える
lpad("abcde",10)

"     abcde"

In [24]:
# 右寄せで10文字に揃える
rpad("abcde",10)

"abcde     "

### 文字列の検索

ある文字列から、特定の文字列を検索し、その位置を返すには `search()` を用いる。
先頭から検索し、最初に現れた位置を範囲で返す。
もし発見されなかった場合には、無意味な範囲（`0:-1`）が返る。
末尾から検索するには、`rsearch()`を利用する。
もし、マッチした先頭位置だけが欲しいなら、`searchindex()` および `rsearchindex()` が利用出来る。

In [25]:
# 文字列内に "abc" が最初に現れる位置を特定する
search("abcDEFabcdef","abc")

1:3

In [26]:
# 最初に出現した位置だけを得る
searchindex("abcDEFabcdef","abc")

1

In [27]:
# 末尾から検索する
rsearch("abcDEFabcdef","abc")

7:9

In [28]:
# 発見できなかった場合
search("abcDEFabcdef","ABC")

0:-1

### 文字列の置換

文字列を置換するには `replace()` を用いる。
上で紹介した `search()` と同じく、最初に与えた文字列から、2番目に与えた文字列を検索する。
そして、3番目に与えた文字列に置き換える。
置換する回数を制約するオプションがある。

In [29]:
# 文字列内の "abc" を、すべて "ABC" に置換する
replace("abcDEFabcdef","abc","ABC")

"ABCDEFABCdef"

In [30]:
# 置換の回数を1回に制限する
replace("abcDEFabcdef","abc","ABC",1)

"ABCDEFabcdef"

### 文字列の置換（高度な利用方法）

置換関数 `replace()` の第3引数には、置き換える文字列だけではなく、関数を指定できる。
マッチした部分文字列をその関数に渡し、その戻り値を出力することができる。

In [31]:
# 文字列内の abc を、すべて "ABC" に置換する
replace("abcDEFabcdef","abc",uppercase)

"ABCDEFABCdef"

### 大文字と小文字の変換

大文字への変換には `uppercase()`、小文字への変換には `lowercase()` を使う。
単語の頭文字だけを大文字にする `titlecase()` もある。

In [32]:
uppercase("abcdefg")

"ABCDEFG"

In [33]:
lowercase("ABCDEFG")

"abcdefg"

In [34]:
titlecase("This is a pen.")

"This Is A Pen."

## 正規表現

Juliaには、Perl相当の正規表現機能が備わっており、検索と置換に利用できる。
上で説明した `search()` や `replace()` は、第2引数に文字列ではなく正規表現を含めることができる。
また、`split()` の区切り文字指定にも正規表現が利用できる。

Perlでは、正規表現は単なる文字列だが、Juliaでは、正規表現を表す特別な文字列（`r""`）を利用する。
以下の例は、正規表現文字列を作り、その型を確認している。

In [35]:
# 文字列に r の接頭辞をつける
r"HOL[0-9]+"

r"HOL[0-9]+"

In [36]:
# これは正規表現の文字列である
typeof(r"HOL[0-9]+")

Regex

### 正規表現による簡単なマッチング

関数 `ismatch()` は、与えた正規表現が、ある文字列内に含まれるかどうかを返す。
結果は `Bool` （マッチすれば`true`、それ以外は`false`）で返る。 
Perlの `=~//` に相当する。

In [37]:
# 与えた正規表現が、文字列内に含まれるかどうかを返す
ismatch(r"[A-Z1-9]+","abcDEF123abcDEF456")

true

### 正規表現に最初にマッチした文字列のみを得る

関数 `match()` は、ある正規表現が文字列にマッチするかどうかを確かめ、その最初のマッチだけを取り出す。
取り出されたデータは構造体であり、詳細な情報を含んでいる。

In [38]:
# マッチしなかった場合は Void または Nothing が返る
m = match(r"abc","DEF")
typeof(m)

Void

In [39]:
# 最初にマッチした部分に関する情報を得る
m = match(r"[A-Z1-9]+","abcDEF123abcDEF456")

RegexMatch("DEF123")

In [40]:
# 最初にマッチした部分文字列
m.match

"DEF123"

In [41]:
# その文字列中の位置（4文字目）
m.offset

4

### 正規表現にマッチした文字列を得る（イテレーター）

文字列中に複数のマッチがある場合、そのすべてを取り出すには2つの方法がある。
一つ目は、イテレーターと呼ばれる仕組みを利用することだ。
関数 `eachmatch()` は、上で示したマッチ構造体の集合を返す。
それを `for` ループにかけると、個々のマッチ情報が得られる。

In [42]:
# マッチした部分に関する情報を得る
em = eachmatch(r"[A-Z1-9]+","abcDEF123abcDEF456")

Base.RegexMatchIterator(r"[A-Z1-9]+", "abcDEF123abcDEF456", false)

In [43]:
# 反復して、それぞれのマッチした部分文字列と開始位置を得る
for i in em
    println(i.match,": ",i.offset)
end

DEF123: 4
DEF456: 13


### 正規表現にマッチした文字列を得る（構造体配列）

関数 `matchall()` は、上の `eachmatch()` とほぼ同じ挙動をするが、マッチ情報をベクトルに入れて返す。
マッチした文字列は、ベクトルと同じく `[]` でアクセスできる。

In [44]:
# マッチした部分に関する情報を得る
ma = matchall(r"[A-Z1-9]+","abcDEF123abcDEF456")

2-element Array{SubString{String},1}:
 "DEF123"
 "DEF456"

In [45]:
# 要素にアクセスすることで、マッチ部分行列を得られる
ma[1:end]

2-element Array{SubString{String},1}:
 "DEF123"
 "DEF456"

In [46]:
# 開始位置は、各要素を参照することで得られる
ma[2].offset

12

### 正規表現による置換

ここでは、Perlで頻出する `s///g` による置換をJuliaで行う方法について説明する。
使う関数は上で説明した `replace()` である。
基本的な置換方法は、正規表現なしの場合と同じである。

In [47]:
# 引数の順序に注意
# DEF123とDEF456がマッチし、それが _ に置き換わる
replace("abcDEF123abcDEF456",r"[A-Z1-9]+","_")

"abc_abc_"

マッチした部分を置換時に再利用する場合は、Perlと同様の手順を取る。
まず、正規表現の中でのちに参照したい部分を `()` でくくる。
次に、置換文字列を `s""` で定義し、最初にマッチした文字列を `\1`、2番目にマッチした文字列を `\2` として参照する。

In [48]:
# 最初の3文字を\1、次の1文字を\2、残りの数値列を\3として参照する
# それを並び替えることで、`F`を末尾に移動している
replace("XYZF0123456789",r"([A-Z][A-Z][A-Z])(.)([0-9]*)",s"\1\3\2")

"XYZ0123456789F"

上の例はPythonスタイルの正規表現で書き直すことができる。
この場合、正規表現内でのちほど参照したい部分を `(?)` でくくり、その中に適当なラベルを `<>` で指定する。
次に、置換文字列を `s""` で定義し、その中で前述のラベルを `\g<>` で参照する。
前述のように `\1` のような表記も利用できる。
上の例と全く同じ例を、Pythonスタイルで書き直す。

In [49]:
# 最初の3文字にはcode、次の1文字にはcategory、最後の数値列にはnumberというラベルをつける
# 参照する場合には、各ラベルを \g<> で指定する
replace("XYZF0123456789",
        r"(?<code>[A-Z][A-Z][A-Z])(?<category>.)(?<number>[0-9]*)",
        s"\g<code>\g<number>\g<category>")

"XYZ0123456789F"